In [1]:
import pandas as pd
import urllib

import numpy as np

import json

from tqdm.autonotebook import tqdm

#%matplotlib inline

tqdm.pandas()

import dask.dataframe as dd

from dask.multiprocessing import get
from dask.diagnostics import ProgressBar

from datetime import datetime
import matplotlib.pyplot as plt

from IPython.display import display


/tmp/ipykernel_177493/3514129619.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
import urllib3

In [3]:
http = urllib3.PoolManager()

In [5]:
from config_batch import * 

# Functions

In [562]:
ws_hostname = "127.0.1.1"
ws_hostname = "172.27.0.64"

# ws_hostname = "192.168.1.3"

'test 1 '

In [7]:
def call_ws_old(addr_data, check_result=True, structured_osm=False): #lg = "en,fr,nl"
    t = datetime.now()
    
    params = urllib.parse.urlencode({"street": addr_data[street_field],
                                     "housenumber": addr_data[housenbr_field],
                                     "city": addr_data[city_field],
                                     "postcode": addr_data[postcode_field],
                                     "country": addr_data[country_field],
                                     "check_result" : "yes" if check_result else "no",
                                     "struct_osm" : "yes" if structured_osm else "no"
                                    })
    url = f"http://{ws_hostname}:5000/search/?{params}"
    
#     print(url)
    try:
        with urllib.request.urlopen(url) as response:
            res = response.read()
            
            res = json.loads(res)
#             print(res)
            res["time"] = (datetime.now() - t).total_seconds()
            return res
    except Exception as e:
        return str(e)
    

In [588]:
def call_ws(addr_data, check_result=True, structured_osm=False): #lg = "en,fr,nl"
    t = datetime.now()
    
    r = http.request(
    'POST',
    f'http://{ws_hostname}:5000/search',
    fields= { 
         "street": addr_data[street_field],
         "housenumber": addr_data[housenbr_field],
         "city": addr_data[city_field],
         "postcode": addr_data[postcode_field],
         "country": addr_data[country_field],
         "check_result" : "yes" if check_result else "no",
         "struct_osm" : "yes" if structured_osm else "no"
                                    
    })
    
    if r.status == 204:
        print("No result!")
        return
    elif r.status == 200:
        try:
            print(r.data.decode('utf-8'))

            res = json.loads(r.data.decode('utf-8'))
            res["time"] = (datetime.now() - t).total_seconds()
        except ValueError as ve:

            print("Cannot decode result:")
            print(ve)
            print(r.data.decode('utf-8'))
            return r.data
        return res
    else: 
        print(f"Unknown return code: {r.status} ")



In [595]:
def call_ws_batch(addr_data, mode="geo", with_rejected=False, check_result=True, structured_osm=False): #lg = "en,fr,nl"
#     print(addr_data)
#     print(addr_data.shape)
#     print()
    file_data = addr_data.rename(columns = {
        street_field : "street",
        housenbr_field: "housenumber",
        postcode_field: "postcode",
        city_field: "city",
        country_field: "country",
        addr_key_field : "addr_key"
    }).to_csv(index=False)
    
    r = http.request(
    'POST',
    f'http://{ws_hostname}:5000/batch',
    fields= { 
        'media': ('addresses.csv', file_data),
        'mode': mode,
        "with_rejected": "yes" if with_rejected else "no",
        "check_result" : "yes" if check_result else "no",
        "struct_osm"   : "yes" if structured_osm else "no",
        #"extra_house_nbr": "no"
    })
    
    try:
        res = pd.DataFrame(json.loads(r.data.decode('utf-8')))
    except ValueError as ve:
        
        print("Cannot decode result:")
        print(ve)
        print(r.data.decode('utf-8'))
     
        return r.data
#     display(res)
    return res

In [482]:
def expand_json(addresses):
    addresses["status"]= addresses.json.apply(lambda d: "error" if "error" in d else "match" if "match" in d else "no_result")
    addresses["time"]  = addresses.json.apply(lambda d: d["time"])

    addresses["timing"]  = addresses.json.apply(lambda d: d["timing"] if "timing" in d else {})
    
    for f in ["method", "place_rank"]:
        addresses[f]= addresses.json.apply(lambda d: d["match"][0][f] if len(d)>0 and "match" in d else "none")
    
    for field in ["street", "number", "postcode", "city"]:
        addresses["addr_out_"+field]= addresses.json.apply(lambda d: d["match"][0]["addr_out_"+field] if len(d)>0 and "match" in d else "")
    return 

# Calls

## Single address calls

In [589]:
res=call_ws({street_field:   "qzg qztqz", \
         housenbr_field: "20",\
         city_field:     "Saint-Gillesazazazza",\
         postcode_field: "5160",\
         country_field:  "Belgique"}, check_result=False, structured_osm=False)
res

No result!


## Batch calls (row by row)

In [553]:
# KBO dataset
addresses = get_addresses("address.csv.gz")
addresses = addresses.sample(10, replace=False).copy()
addresses["addr_key"] = addresses["addr_key"].astype(str)
addresses

,addr_key,CountryFR,postcode,city,street,housenumber
80,3072328,Belgique,4000,Liège,Rue des Bergers,8
139,1536868,Belgique,1640,Sint-Genesius-Rode,Avenue des Chênes,23
442,2722384,Belgique,2840,Rumst,Steenberghoekstraat,24
305,2653276,Belgique,9050,Gent,Brusselsesteenweg,500
367,875298,Belgique,8800,Roeselare,Steenstraat,54
677,1535976,Belgique,2640,Mortsel,Edmond Thieffrylaan,98
639,2727011,Belgique,9300,Aalst,Gentse steenweg,338
958,2582872,Belgique,2140,Antwerpen,Jozef Nuytsstraat,22
528,603081,Belgique,1500,Halle,Sint-Rochusstraat,99
366,1139151,Belgique,1421,Braine-l'Alleud,Rue Chavée,24


In [459]:
# addresses[addresses.addr_key.duplicated()]

In [407]:
addresses = get_addresses("../GISAnalytics/data/geocoding/resto_1000_sample.csv.gz")
# addresses = get_addresses("../GISAnalytics/data/geocoding/best_1000_sample.csv.gz")
addresses["addr_key"] = addresses.index.astype(str)
addresses

,street,housenumber,city,postcode,CountryFR,addr_key
0,Chaussée de Namur,198,LEUZE,5310,Belgique,0
1,Rue de la Halle,11,NAMUR,5000,Belgique,1
2,Avenue Jacques Sermon,38,GANSHOREN,1083,Belgique,2
3,rue du Centre,200,WAIMES,4950,Belgique,3
4,Kleine Waterstraat,9,HULSHOUT,2235,Belgique,4
...,...,...,...,...,...,...
995,Rue Americaine,90,IXELLES,1050,Belgique,995
996,Kloosterstraat,60,EKEREN,2180,Belgique,996
997,Markt,13,BALEN,2490,Belgique,997
998,Lange Lobroekstraat,65,ANVERS,2060,Belgique,998


In [408]:
addresses = pd.concat([
    get_addresses("../GISAnalytics/data/geocoding/resto_1000_sample.csv.gz"),
    get_addresses("../GISAnalytics/data/geocoding/best_1000_sample.csv.gz"),
    get_addresses("address.csv.gz")])
addresses = addresses.reset_index(drop=True)
addresses["addr_key"] = addresses.index.astype(str)

In [312]:
# addresses = addresses[addresses.addr_key.isin([ "2078829"])]#"1622",
# addresses


### Simple way

In [590]:
addresses_seq = addresses.copy()

t = datetime.now()
addresses_seq["json"] = addresses_seq.fillna("").progress_apply(call_ws, check_result=False, structured_osm=False, axis=1)
tot_time = (datetime.now() - t).total_seconds()
print(f"{tot_time:.2f} seconds, {addresses_seq.shape[0]/tot_time:.2f} it/s")
# KBO dataset:
# Normal mode: 128.78 seconds, 7.77 it/s
# Fastmode:     68.80 seconds, 14.54 it/s

#Resto dataset: 
# Normal mode: 145.73 seconds, 6.86 it/s
# Fast mode:    82.99 seconds, 12.05 it/s

# Best dataset:
# Normal mode: 108.53 seconds, 9.21 it/s
# Fast mode: 37.44 seconds, 26.71 it/s

  0%|          | 0/10 [00:00<?, ?it/s]

{"match": [{"method": "fast", "display_name": "8, Rue des Bergers, Thier-\u00e0-Li\u00e8ge, Jupille-sur-Meuse, Li\u00e8ge, Wallonie, 4000, Belgi\u00eb / Belgique / Belgien", "place_id": 1617602, "lat": "50.66238025", "lon": "5.591573123340931", "place_rank": 30, "addr_out_street": "Rue des Bergers", "addr_out_city": "Jupille-sur-Meuse", "addr_out_number": "8", "addr_out_country": "Belgi\u00eb / Belgique / Belgien", "addr_out_postcode": "4000", "in_house_nbr": "8", "lpost_house_nbr": "8", "lpost_unit": "", "osm_addr_in": "Rue des Bergers, 8, 4000 Li\u00e8ge, Belgique"}]}

{"match": [{"method": "fast", "display_name": "23, Eikenlaan - Avenue des Ch\u00eanes, Sint-Anna - Sainte-Anne, Sint-Genesius-Rode, Halle-Vilvoorde, Vlaams-Brabant, Vlaanderen, 1640, Belgi\u00eb / Belgique / Belgien", "place_id": 2244309, "lat": "50.75512845", "lon": "4.383524401557887", "place_rank": 30, "addr_out_street": "Eikenlaan - Avenue des Ch\u00eanes", "addr_out_city": "Sint-Genesius-Rode", "addr_out_number": 

In [591]:
expand_json(addresses_seq)
addresses_seq

,addr_key,CountryFR,postcode,city,street,housenumber,json,status,time,timing,method,place_rank,addr_out_street,addr_out_number,addr_out_postcode,addr_out_city
80,3072328,Belgique,4000,Liège,Rue des Bergers,8,"{'match': [{'method': 'fast', 'display_name': ...",match,0.034936,{},fast,30,Rue des Bergers,8,4000,Jupille-sur-Meuse
139,1536868,Belgique,1640,Sint-Genesius-Rode,Avenue des Chênes,23,"{'match': [{'method': 'fast', 'display_name': ...",match,0.032850,{},fast,30,Eikenlaan - Avenue des Chênes,23,1640,Sint-Genesius-Rode
442,2722384,Belgique,2840,Rumst,Steenberghoekstraat,24,"{'match': [{'method': 'fast', 'display_name': ...",match,0.058804,{},fast,30,Steenberghoekstraat,24,2840,Rumst
305,2653276,Belgique,9050,Gent,Brusselsesteenweg,500,"{'match': [{'method': 'fast', 'display_name': ...",match,0.160102,{},fast,26,Brusselsesteenweg,,9050,Gentbrugge
367,875298,Belgique,8800,Roeselare,Steenstraat,54,"{'match': [{'method': 'fast', 'display_name': ...",match,0.057165,{},fast,30,Steenstraat,54,8800,Roeselare
677,1535976,Belgique,2640,Mortsel,Edmond Thieffrylaan,98,"{'match': [{'method': 'fast', 'display_name': ...",match,0.026430,{},fast,30,Edmond Thieffrylaan,98,2640,Mortsel
639,2727011,Belgique,9300,Aalst,Gentse steenweg,338,"{'match': [{'method': 'fast', 'display_name': ...",match,0.062004,{},fast,30,Gentsesteenweg,338,9300,Aalst
958,2582872,Belgique,2140,Antwerpen,Jozef Nuytsstraat,22,"{'match': [{'method': 'fast', 'display_name': ...",match,0.030361,{},fast,30,Jozef Nuytsstraat,22,2140,Borgerhout
528,603081,Belgique,1500,Halle,Sint-Rochusstraat,99,"{'match': [{'method': 'fast', 'display_name': ...",match,0.107681,{},fast,30,Sint-Rochusstraat,99,1500,Halle
366,1139151,Belgique,1421,Braine-l'Alleud,Rue Chavée,24,"{'match': [{'method': 'fast', 'display_name': ...",match,0.035543,{},fast,26,Rue Chavée,,1421,Braine-l'Alleud


In [488]:
# addresses_seq.iloc[0].json

In [510]:
addresses_seq.method.value_counts()

fast                                   847
regex[init]                            103
libpostal+regex[lpost]+photon           29
libpostal+regex[lpost]                  14
nostreet                                 5
libpostal+regex[lpost]+photon+nonum      1
photon                                   1
Name: method, dtype: int64

In [433]:
addresses_seq

,street,housenumber,city,postcode,CountryFR,addr_key,json,status,time,timing,method,addr_out_street,addr_out_number,addr_out_postcode,addr_out_city
0,Chaussée de Namur,198,LEUZE,5310,Belgique,0,"{'match': [{'SIM_city': 1.0, 'SIM_house_nbr': ...",match,0.190694,"{'extra_hn': 13.703999999999999, 'format_res':...",orig,Chaussée de Namur,198,5310,Leuze
1,Rue de la Halle,11,NAMUR,5000,Belgique,1,"{'match': [{'SIM_city': 1.0, 'SIM_house_nbr': ...",match,0.228016,"{'extra_hn': 13.896, 'format_res': 6.432, 'glo...",orig,Rue de la Halle,,5000,Namur
2,Avenue Jacques Sermon,38,GANSHOREN,1083,Belgique,2,"{'match': [{'SIM_city': 1.0, 'SIM_house_nbr': ...",match,0.254582,"{'extra_hn': 13.880999999999998, 'format_res':...",orig,Avenue Jacques Sermon - Jacques Sermonlaan,38,1083,Ganshoren
3,rue du Centre,200,WAIMES,4950,Belgique,3,"{'match': [{'SIM_city': 1.0, 'SIM_house_nbr': ...",match,0.230515,"{'extra_hn': 14.522, 'format_res': 6.391, 'glo...",orig,Rue du Centre,,4950,Waimes
4,Kleine Waterstraat,9,HULSHOUT,2235,Belgique,4,"{'match': [{'SIM_city': 1.0, 'SIM_house_nbr': ...",match,0.188431,"{'extra_hn': 13.521999999999998, 'format_res':...",orig,Kleine Waterstraat,9,2235,Hulshout
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,Rue du Wainage(VEL),62,Sambreville,5060,Belgique,2995,"{'match': [{'SIM_city': 1.0, 'SIM_house_nbr': ...",match,0.247127,"{''t&p > transformer > libpostal': 0.0, ''t&p ...",regex[init],Rue du Wainage,62,5060,Sambreville
2996,Chaussée de Tournai(R-C),4,Tournai,7520,Belgique,2996,"{'match': [{'SIM_city': 0.125, 'SIM_house_nbr'...",match,0.237380,"{''t&p > transformer > libpostal': 0.0, ''t&p ...",orig,Chaussée de Tournai,4,7641,Bruyelle
2997,Rue Louis Caty(B),32,Saint-Ghislain,7331,Belgique,2997,"{'match': [{'SIM_city': 1.0, 'SIM_house_nbr': ...",match,0.242513,"{''t&p > transformer > libpostal': 0.0, ''t&p ...",orig,Rue Louis Caty,,7331,Saint-Ghislain
2998,Drève des Bouleaux,5,Braine-le-Comte,7090,Belgique,2998,"{'match': [{'SIM_city': 1.0, 'SIM_house_nbr': ...",match,0.211866,"{''t&p > transformer > libpostal': 0.0, ''t&p ...",orig,Drève des Bouleaux,,7090,Braine-le-Comte


In [432]:
addresses_seq[addresses_seq.status == "rejected"]

,street,housenumber,city,postcode,CountryFR,addr_key,json,status,time,timing,method,addr_out_street,addr_out_number,addr_out_postcode,addr_out_city
611,"Rue haute,",[na],KOEKELBERG,1000,Belgique,611,"{'rejected': [{'SIM_city': 0.0, 'SIM_house_nbr...",rejected,2.698123,{''t&p > transformer > libpostal': 575.0659999...,none,,,,
784,route de Balmoral,35,SART-LEZ-SPA,4845,Belgique,784,"{'rejected': [], 'timing': {''t&p > transforme...",rejected,1.645492,"{''t&p > transformer > libpostal': 396.381, ''...",none,,,,


In [402]:
# addresses_seq[addresses_seq.method.str.contains("photon")]

In [392]:
addresses_seq.json.apply(lambda json_rec: [ r["dist_to_match"] for r in json_rec['reject']] if "reject" in json_rec else []).iloc[0:60]

0                                                    []
1                                                    []
2                                 [0.15477450367065837]
3                                                    []
4     [1.828569510773582, 3.273126517453961, 2.71100...
5                                                    []
6                                  [1.1139986052590796]
7                                                    []
8                                                    []
9                                                    []
10                                                   []
11                                                   []
12                                                   []
13                                                   []
14                               [0.007063384220609743]
15                                                   []
16                                                   []
17                                              

In [394]:
addresses_seq.iloc[6].json

{'match': [{'addr_out_city': 'Gent',
   'addr_out_country': 'België / Belgique / Belgien',
   'addr_out_number': '',
   'addr_out_postcode': '9000',
   'addr_out_street': 'Krijgslaan',
   'display_name': 'Krijgslaan, Sint-Pieters-Aalst, Gent, Oost-Vlaanderen, Vlaanderen, 9000, België / Belgique / Belgien',
   'in_house_nbr': '74',
   'lat': '51.0348811',
   'lon': '3.7201409',
   'lpost_house_nbr': '74',
   'lpost_unit': '',
   'method': 'fast',
   'osm_addr_in': 'Krijgslaan, 74, 9000 Gent, Belgique',
   'place_id': 984433,
   'place_rank': 26}],
 'reject': [{'addr_out_city': 'Gent',
   'addr_out_country': 'België / Belgique / Belgien',
   'addr_out_number': '',
   'addr_out_postcode': '9000',
   'addr_out_street': 'Krijgslaan',
   'display_name': 'Krijgslaan, Stationsbuurt Zuid, Gent, Oost-Vlaanderen, Vlaanderen, 9000, België / Belgique / Belgien',
   'dist_to_match': 1.1139986052590796,
   'lat': '51.0264173',
   'lon': '3.7116547',
   'method': 'fast',
   'place_id': 313241,
   'pla

In [150]:
addresses_seq.method.value_counts()

fast                             979
libpostal+regex[lpost]+photon     11
nostreet                           7
regex[init]                        2
nonum                              1
Name: method, dtype: int64

### Using Dask

In [229]:
addresses_dask = addresses.copy()

In [491]:
t = datetime.now()
dd_addresses = dd.from_pandas(addresses_dask, npartitions=8)

dask_task = dd_addresses.apply(call_ws, check_result=False, meta=('x', 'str'), axis=1)

with ProgressBar(): 
    addresses_dask["json"] = dask_task.compute()
    
tot_time = (datetime.now() - t).total_seconds()
print(f"{tot_time:.2f} seconds, {addresses_dask.shape[0]/tot_time:.2f} it/s")
# KBO dataset:
# Normal mode: 24.52 seconds, 40.79 it/s
# Fastmode:    15.81 seconds, 63.27 it/s


# Resto dataset:
# Normal mode: 27.86 seconds, 35.89 it/s
# Fast mode:   18.44 seconds, 54.23 it/s

# Best dataset: 
# Normal mode: 16.11 seconds, 62.07 it/s
# Fast mode:    9.76 seconds, 102.42 it/s

[########################################] | 100% Completed | 17.1s
17.19 seconds, 58.16 it/s


In [90]:
# 1000, 1 worker: 4m18
# 4 workers, npart=4 : 1m20
# 8 workers, npart=4 : 1m20
# 8 workers, npart=8 : 44s

# with checker=False:
# 8 workers, npart=8 : 24s


In [492]:
expand_json(addresses_dask)
addresses_dask

,street,housenumber,city,postcode,CountryFR,addr_key,json,status,time,timing,method,addr_out_street,addr_out_number,addr_out_postcode,addr_out_city,place_rank
0,Chaussée de Namur,198,LEUZE,5310,Belgique,0,"{'match': [{'addr_out_city': 'Leuze', 'addr_ou...",match,0.032007,"{''t&p > transformer > libpostal': 0.0, ''t&p ...",fast,Chaussée de Namur,198,5310,Leuze,30
1,Rue de la Halle,11,NAMUR,5000,Belgique,1,"{'match': [{'addr_out_city': 'Namur', 'addr_ou...",match,0.033146,"{''t&p > transformer > libpostal': 0.0, ''t&p ...",fast,Rue de la Halle,,5000,Namur,26
2,Avenue Jacques Sermon,38,GANSHOREN,1083,Belgique,2,"{'match': [{'addr_out_city': 'Ganshoren', 'add...",match,0.027931,"{''t&p > transformer > libpostal': 0.0, ''t&p ...",fast,Avenue Jacques Sermon - Jacques Sermonlaan,38,1083,Ganshoren,30
3,rue du Centre,200,WAIMES,4950,Belgique,3,"{'match': [{'addr_out_city': 'Waimes', 'addr_o...",match,0.066464,"{''t&p > transformer > libpostal': 0.0, ''t&p ...",fast,Rue du Centre,,4950,Waimes,26
4,Kleine Waterstraat,9,HULSHOUT,2235,Belgique,4,"{'match': [{'addr_out_city': 'Hulshout', 'addr...",match,0.025898,"{''t&p > transformer > libpostal': 0.0, ''t&p ...",fast,Kleine Waterstraat,9,2235,Hulshout,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Rue Americaine,90,IXELLES,1050,Belgique,995,{'match': [{'addr_out_city': 'Ixelles - Elsene...,match,0.028916,"{''t&p > transformer > libpostal': 0.0, ''t&p ...",fast,Rue Américaine - Amerikaanse straat,,1050,Ixelles - Elsene,26
996,Kloosterstraat,60,EKEREN,2180,Belgique,996,"{'match': [{'addr_out_city': 'Ekeren', 'addr_o...",match,0.024279,"{''t&p > transformer > libpostal': 0.0, ''t&p ...",fast,Kloosterstraat,60,2180,Ekeren,30
997,Markt,13,BALEN,2490,Belgique,997,"{'match': [{'addr_out_city': 'Balen', 'addr_ou...",match,0.021137,"{''t&p > transformer > libpostal': 0.0, ''t&p ...",fast,Markt,13,2490,Balen,30
998,Lange Lobroekstraat,65,ANVERS,2060,Belgique,998,"{'match': [{'addr_out_city': 'Antwerpen', 'add...",match,0.021730,"{''t&p > transformer > libpostal': 0.0, ''t&p ...",fast,Lange Lobroekstraat,65,2060,Antwerpen,30


In [201]:
addresses_dask.method.value_counts()#.json.loc[550]

fast                             979
libpostal+regex[lpost]+photon     11
nostreet                           7
regex[init]                        2
nonum                              1
Name: method, dtype: int64

In [232]:
mg = addresses_seq[["addr_key", "city", "postcode","street", "housenumber", "addr_out_street", "addr_out_number", "addr_out_postcode", "addr_out_city"]].merge(
    addresses_dask[["addr_key", "city", "postcode","street", "housenumber", "addr_out_street", "addr_out_number", "addr_out_postcode", "addr_out_city"]], how="outer", indicator=True)
if mg.shape[0] == addresses.shape[0]:
    print("Same result in seq and dask run!")
else: 
    print("!!! Not the same result in seq and dask run!")
    

Same result in seq and dask run!


In [233]:
mg[mg._merge != "both"].sort_values("addr_key")


,addr_key,city,postcode,street,housenumber,addr_out_street,addr_out_number,addr_out_postcode,addr_out_city,_merge


## Batch calls (batch WS)

### Single block

In [600]:
t = datetime.now()

addresses_batch = call_ws_batch(addresses[[addr_key_field, 
                                           street_field, housenbr_field, postcode_field, city_field, country_field]], 
                                mode="long", 
                                check_result=True, 
                                structured_osm=True,
                               with_rejected=True)

tot_time = (datetime.now() - t).total_seconds()
print(f"{tot_time:.2f} seconds, {addresses.shape[0]/tot_time:.2f} it/s")
# KBO dataset: 33.94 seconds, 29.46 it/s
# Best:        24.99 seconds, 40.01 it/s
# Resto:       38.33 seconds, 26.09 it/s

addresses_batch

0.53 seconds, 19.02 it/s


,addr_key,street,housenumber,postcode,city,country,osm_addr_in,place_id,lat,lon,...,addr_out_city,addr_out_number,addr_out_country,addr_out_postcode,addr_out_other,method,in_house_nbr,lpost_house_nbr,lpost_unit,reject
0,3072328,Rue des Bergers,8,4000,Liège,Belgique,"Rue des Bergers, 8, 4000 Liège, Belgique",1617602,50.662380,5.591573,...,Jupille-sur-Meuse,8,België / Belgique / Belgien,4000,Thier-à-Liège - Wallonie,orig,8,8,,[]
1,1536868,Avenue des Chênes,23,1640,Sint-Genesius-Rode,Belgique,"Avenue des Chênes, 23, 1640 Sint-Genesius-Rode...",2244309,50.755128,4.383524,...,Sint-Genesius-Rode,23,België / Belgique / Belgien,1640,Vlaanderen - Sint-Anna - Sainte-Anne,orig,23,23,,[]
2,2722384,Steenberghoekstraat,24,2840,Rumst,Belgique,"Steenberghoekstraat, 24, 2840 Rumst, Belgique",676471,51.080606,4.418632,...,Rumst,24,België / Belgique / Belgien,2840,Brandweer voorpost Rumst - Vlaanderen,orig,24,24,,[]
3,2653276,Brusselsesteenweg,500,9050,Gent,Belgique,"Brusselsesteenweg, 500, 9050 Gent, Belgique",926149,51.036814,3.750606,...,Gentbrugge,None,België / Belgique / Belgien,9050,Vlaanderen,orig,500,500,,"[{'index': 4, 'osm_addr_in': 'Brusselsesteenwe..."
4,875298,Steenstraat,54,8800,Roeselare,Belgique,"Steenstraat, 54, 8800 Roeselare, Belgique",2423675,50.954290,3.119699,...,Roeselare,54,België / Belgique / Belgien,8800,Vlaanderen - Beekkwartier,orig,54,54,,[]
5,1535976,Edmond Thieffrylaan,98,2640,Mortsel,Belgique,"Edmond Thieffrylaan, 98, 2640 Mortsel, Belgique",1873639,51.166325,4.449987,...,Mortsel,98,België / Belgique / Belgien,2640,Vlaanderen,orig,98,98,,[]
6,2727011,Gentse steenweg,338,9300,Aalst,Belgique,"Gentse steenweg, 338, 9300 Aalst, Belgique",3549623,50.939927,4.013137,...,Aalst,338,België / Belgique / Belgien,9300,Vlaanderen,orig,338,338,,[]
7,2582872,Jozef Nuytsstraat,22,2140,Antwerpen,Belgique,"Jozef Nuytsstraat, 22, 2140 Antwerpen, Belgique",2120193,51.210073,4.449621,...,Borgerhout,22,België / Belgique / Belgien,2140,Vlaanderen - Borgerhout Extramuros,orig,22,22,,[]
8,603081,Sint-Rochusstraat,99,1500,Halle,Belgique,"Sint-Rochusstraat, 99, 1500 Halle, Belgique",2161717,50.729769,4.241530,...,Halle,99,België / Belgique / Belgien,1500,Vlaanderen - Sint-Rochus,orig,99,99,,[]
9,1139151,Rue Chavée,24,1421,Braine-l'Alleud,Belgique,"Rue Chavée, 24, 1421 Braine-l'Alleud, Belgique",570124,50.666250,4.355740,...,Braine-l'Alleud,None,België / Belgique / Belgien,1421,Wallonie,orig,24,24,,[]


In [597]:
addresses_batch

,addr_key,street,housenumber,postcode,city,country,osm_addr_in,place_id,lat,lon,...,addr_out_city,addr_out_number,addr_out_country,addr_out_postcode,addr_out_other,method,in_house_nbr,lpost_house_nbr,lpost_unit,reject
0,3072328,Rue des Bergers,8,4000,Liège,Belgique,"Rue des Bergers, 8, 4000 Liège, Belgique",1617602,50.662380,5.591573,...,Jupille-sur-Meuse,8,België / Belgique / Belgien,4000,Thier-à-Liège - Wallonie,orig,8,8,,[]
1,1536868,Avenue des Chênes,23,1640,Sint-Genesius-Rode,Belgique,"Avenue des Chênes, 23, 1640 Sint-Genesius-Rode...",2244309,50.755128,4.383524,...,Sint-Genesius-Rode,23,België / Belgique / Belgien,1640,Vlaanderen - Sint-Anna - Sainte-Anne,orig,23,23,,[]
2,2722384,Steenberghoekstraat,24,2840,Rumst,Belgique,"Steenberghoekstraat, 24, 2840 Rumst, Belgique",676471,51.080606,4.418632,...,Rumst,24,België / Belgique / Belgien,2840,Brandweer voorpost Rumst - Vlaanderen,orig,24,24,,[]
3,2653276,Brusselsesteenweg,500,9050,Gent,Belgique,"Brusselsesteenweg, 500, 9050 Gent, Belgique",926149,51.036814,3.750606,...,Gentbrugge,None,België / Belgique / Belgien,9050,Vlaanderen,orig,500,500,,"[{'osm_addr_in': 'Brusselsesteenweg, 500, 9050..."
4,875298,Steenstraat,54,8800,Roeselare,Belgique,"Steenstraat, 54, 8800 Roeselare, Belgique",2423675,50.954290,3.119699,...,Roeselare,54,België / Belgique / Belgien,8800,Vlaanderen - Beekkwartier,orig,54,54,,[]
5,1535976,Edmond Thieffrylaan,98,2640,Mortsel,Belgique,"Edmond Thieffrylaan, 98, 2640 Mortsel, Belgique",1873639,51.166325,4.449987,...,Mortsel,98,België / Belgique / Belgien,2640,Vlaanderen,orig,98,98,,[]
6,2727011,Gentse steenweg,338,9300,Aalst,Belgique,"Gentse steenweg, 338, 9300 Aalst, Belgique",3549623,50.939927,4.013137,...,Aalst,338,België / Belgique / Belgien,9300,Vlaanderen,orig,338,338,,[]
7,2582872,Jozef Nuytsstraat,22,2140,Antwerpen,Belgique,"Jozef Nuytsstraat, 22, 2140 Antwerpen, Belgique",2120193,51.210073,4.449621,...,Borgerhout,22,België / Belgique / Belgien,2140,Vlaanderen - Borgerhout Extramuros,orig,22,22,,[]
8,603081,Sint-Rochusstraat,99,1500,Halle,Belgique,"Sint-Rochusstraat, 99, 1500 Halle, Belgique",2161717,50.729769,4.241530,...,Halle,99,België / Belgique / Belgien,1500,Vlaanderen - Sint-Rochus,orig,99,99,,[]
9,1139151,Rue Chavée,24,1421,Braine-l'Alleud,Belgique,"Rue Chavée, 24, 1421 Braine-l'Alleud, Belgique",570124,50.666250,4.355740,...,Braine-l'Alleud,None,België / Belgique / Belgien,1421,Wallonie,orig,24,24,,[]


In [536]:
json.loads(json.loads(addresses_batch.decode("utf8")))

[{'addr_key': '3011419',
  'street': 'Rue Emile Francqui',
  'housenumber': '9',
  'postcode': '1435',
  'city': 'Mont-Saint-Guibert',
  'country': 'Belgique',
  'osm_addr_in': 'Rue Emile Francqui, 9, 1435 Mont-Saint-Guibert, Belgique',
  'place_id': 1088478.0,
  'lat': 50.6590017,
  'lon': 4.623283783,
  'display_name': '9, Rue Émile Francqui, Corbais, Mont-Saint-Guibert, Nivelles, Brabant wallon, Wallonie, 1435, België / Belgique / Belgien',
  'namedetails': '',
  'place_rank': 30.0,
  'category': 'place',
  'type': 'house',
  'SIM_street_which': None,
  'addr_out_street': 'Rue Émile Francqui',
  'addr_out_city': 'Corbais',
  'addr_out_number': '9',
  'addr_out_country': 'België / Belgique / Belgien',
  'addr_out_postcode': '1435',
  'addr_out_other': 'Wallonie',
  'retry_on_26': None,
  'method': 'orig',
  'in_house_nbr': '9',
  'lpost_house_nbr': '9',
  'lpost_unit': '',
  'rejected': None},
 {'addr_key': '1037433',
  'street': "Rue Grand'Ville",
  'housenumber': '97',
  'postcode'

In [534]:
addresses_batch

b'"[{\\"addr_key\\":\\"3011419\\",\\"street\\":\\"Rue Emile Francqui\\",\\"housenumber\\":\\"9\\",\\"postcode\\":\\"1435\\",\\"city\\":\\"Mont-Saint-Guibert\\",\\"country\\":\\"Belgique\\",\\"osm_addr_in\\":\\"Rue Emile Francqui, 9, 1435 Mont-Saint-Guibert, Belgique\\",\\"place_id\\":1088478.0,\\"lat\\":50.6590017,\\"lon\\":4.623283783,\\"display_name\\":\\"9, Rue \\\\u00c9mile Francqui, Corbais, Mont-Saint-Guibert, Nivelles, Brabant wallon, Wallonie, 1435, Belgi\\\\u00eb \\\\/ Belgique \\\\/ Belgien\\",\\"namedetails\\":\\"\\",\\"place_rank\\":30.0,\\"category\\":\\"place\\",\\"type\\":\\"house\\",\\"SIM_street_which\\":null,\\"addr_out_street\\":\\"Rue \\\\u00c9mile Francqui\\",\\"addr_out_city\\":\\"Corbais\\",\\"addr_out_number\\":\\"9\\",\\"addr_out_country\\":\\"Belgi\\\\u00eb \\\\/ Belgique \\\\/ Belgien\\",\\"addr_out_postcode\\":\\"1435\\",\\"addr_out_other\\":\\"Wallonie\\",\\"retry_on_26\\":null,\\"method\\":\\"orig\\",\\"in_house_nbr\\":\\"9\\",\\"lpost_house_nbr\\":\\"9\\"

In [494]:
addresses[addresses.addr_key.duplicated(keep=False)]

,addr_key,CountryFR,postcode,city,street,housenumber


In [463]:
addresses_batch

,addr_key,street,housenumber,postcode,city,country,osm_addr_in,place_id,lat,lon,...,addr_out_number,addr_out_country,addr_out_postcode,addr_out_other,retry_on_26,method,in_house_nbr,lpost_house_nbr,lpost_unit,rejected
0,1683263,Rumbeeksegravier,166,8800,Roeselare,Belgique,"Rumbeeksegravier, 166, 8800 Roeselare, Belgique",2892854.0,50.943255,3.162886,...,166,België / Belgique / Belgien,8800,Vlaanderen - Schaapbrugge,None,orig,166,166,,None
1,1129778,Hazewindstraat,7,8610,Kortemark,Belgique,"Hazewindstraat, 7, 8610 Kortemark, Belgique",484591.0,51.004002,2.992377,...,None,België / Belgique / Belgien,8610,Vlaanderen,None,orig,7,7,,None
2,458847,Vondelstraat,64,8310,Brugge,Belgique,"Vondelstraat, 64, 8310 Brugge, Belgique",2626769.0,51.205519,3.247764,...,64,België / Belgique / Belgien,8310,Vlaanderen,None,orig,64,64,,None
3,2543239,Sterrebosdreef,44,8800,Roeselare,Belgique,"Sterrebosdreef, 44, 8800 Roeselare, Belgique",1342520.0,50.925741,3.133248,...,44,België / Belgique / Belgien,8800,Vlaanderen - Vijfwegen,None,orig,44,44,,None
4,1819581,Rue de la Station,108,1410,Waterloo,Belgique,"Rue de la Station, 108, 1410 Waterloo, Belgique",389704.0,50.717132,4.393372,...,None,België / Belgique / Belgien,1410,Wallonie - Petit Paris,None,orig,108,108,,"[{'osm_addr_in': 'Rue de la Station, 108, 1410..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2266697,Féronstrée,149,4000,Liège,Belgique,"Féronstrée, 149, 4000 Liège, Belgique",344010.0,50.645893,5.577066,...,None,België / Belgique / Belgien,4000,Wallonie - Féronstrée,None,orig,149,149,,None
96,1579674,Sur les Heids de Ninane,42,4050,Chaudfontaine,Belgique,"Sur les Heids de Ninane, 42, 4050 Chaudfontain...",3089576.0,50.583092,5.631375,...,42,België / Belgique / Belgien,4050,Wallonie,None,orig,42,42,,None
97,2753787,Rue du Trieu(D),144,7711,Mouscron,Belgique,"Rue du Trieu(D), 144, 7711 Mouscron, Belgique",403137.0,50.723726,3.314117,...,None,België / Belgique / Belgien,7711,Wallonie,None,orig,144,144,,None
98,986882,Soerveldweg,32,3650,Dilsen-Stokkem,Belgique,"Soerveldweg, 32, 3650 Dilsen-Stokkem, Belgique",1274421.0,51.026230,5.724147,...,None,België / Belgique / Belgien,3650,Vlaanderen,None,orig,32,32,,"[{'osm_addr_in': 'Soerveldweg, 32, 3650 Dilsen..."


In [242]:
mg = addresses_seq[[ "city", "postcode","street", "housenumber", "method", "addr_out_street", "addr_out_number", "addr_out_postcode", "addr_out_city", "addr_key"]].fillna("").replace("fast", "orig").merge(
    addresses_batch[["city", "postcode","street", "housenumber", "method", "addr_out_street", "addr_out_number", "addr_out_postcode", "addr_out_city", "addr_key"]].fillna(""), how="outer", indicator=True)
if mg[mg._merge == "both"].shape[0] == addresses.shape[0]:
    print("Same result in seq and dask run!")
else: 
    print("!!! Not the same result in seq and dask run!")
    

!!! Not the same result in seq and dask run!


In [243]:
mg[mg._merge != "both"]

,city,postcode,street,housenumber,method,addr_out_street,addr_out_number,addr_out_postcode,addr_out_city,addr_key,_merge
46,LIVES-SUR-MEUSE,5101,Chaussée de Marche,563,none,,,,,46,left_only
784,SART-LEZ-SPA,4845,route de Balmoral,35,none,,,,,784,left_only


In [213]:
# Geocode + address
call_ws_batch(addresses[[addr_key_field, street_field, housenbr_field, postcode_field, city_field, country_field]], mode="geo", check_result=False) 

,addr_key,lat,lon,place_rank,method
0,0,50.956670,3.641552,26.0,orig
1,1,50.911196,4.715074,30.0,orig
2,2,50.826581,4.302950,30.0,orig
3,3,51.184610,3.567174,26.0,orig
4,4,51.028284,4.603456,30.0,orig
...,...,...,...,...,...
995,233,50.848531,3.537555,14.0,nostreet
996,874,50.848531,3.537555,14.0,nostreet
997,207,51.057013,3.799241,21.0,nostreet
998,460,50.816471,3.339828,21.0,nostreet


In [249]:
# Geocode + address
call_ws_batch(addresses[[addr_key_field, street_field, housenbr_field, postcode_field, city_field, country_field]], mode="short", check_result=False) 

,addr_key,lat,lon,place_rank,method,place_id,addr_out_street,addr_out_number,in_house_nbr,lpost_house_nbr,lpost_unit,addr_out_postcode,addr_out_city,addr_out_country
0,2762939,51.211615,4.649820,30.0,orig,1032308.0,Kriekenlaan,22,22,22,,2240,Zandhoven,België / Belgique / Belgien
1,2253494,51.059548,2.870803,21.0,nostreet,4005124.0,None,None,*,,,8600,Diksmuide,België / Belgique / Belgien
2,1764140,50.924678,3.788056,26.0,orig,436186.0,Poststraat,None,19,19,,9860,Issegem,België / Belgique / Belgien
3,87070,51.351333,3.285724,30.0,orig,3675084.0,Sylvain Dupuisstraat,24;26;28,26,26,,8300,Knokke-Heist,België / Belgique / Belgien
4,2261339,50.977155,5.690444,26.0,orig,379663.0,Oude Baan,None,338,,m 338,3630,Maasmechelen,België / Belgique / Belgien
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2720764,50.462361,4.596458,30.0,regex[init],2657266.0,Rue du Wainage,62,62,62,,5060,Sambreville,België / Belgique / Belgien
996,386196,50.639445,3.349058,26.0,orig,1983533.0,Chaussée de Tournai,None,4,4,,7520,Ramegnies-Chin,België / Belgique / Belgien
997,1474962,50.479737,3.838976,26.0,orig,432987.0,Rue Louis Caty,None,32,32,,7331,Saint-Ghislain,België / Belgique / Belgien
998,2424889,50.606398,4.159495,26.0,orig,379561.0,Drève des Bouleaux,None,5,5,,7090,Braine-le-Comte,België / Belgique / Belgien


In [360]:
# Geocode + address, with rejected addresses
call_ws_batch(addresses, mode="long", with_reject=True)

,error
0,Field 'addr_key' cannot contain duplicated val...


### Batch blocs

In [359]:
# addresses = addresses.sample(10000, replace=True)
# addresses = addresses.reset_index(drop=True)
# addresses["addr_key"]= addresses.index.astype(str)

In [352]:
t = datetime.now()

nb_threads=8

chunks = np.array_split(addresses, nb_threads) # addresses.shape[0]//100)

print(f"{len(chunks)} chunks on {nb_threads} threads")

import multiprocess as mp

p = mp.Pool(nb_threads)

def f(chunk):
    return call_ws_batch(chunk, mode="long", 
                        check_result=False, 
                        structured_osm=False)

with p:
     res= list(tqdm(p.imap(f, chunks), total=len(chunks)))
    
addresses_batch2 = pd.concat(res).reset_index(drop=True)

tot_time = (datetime.now() - t).total_seconds()
print(f"{tot_time:.2f} seconds, {addresses.shape[0]/tot_time:.2f} it/s")
# KBO:    9.28 seconds, 107.72 it/s
# Best:   6.88 seconds, 145.43 it/s
# Resto: 11.79 seconds,  84.85 it/s

20 chunks on 20 threads


  0%|          | 0/20 [00:00<?, ?it/s]

23.39 seconds, 128.27 it/s


In [361]:
# addresses_batch2

In [317]:
mg = addresses_seq[[ "city", "postcode","street", "housenumber", "method", "addr_out_street", "addr_out_number", "addr_out_postcode", "addr_out_city", "addr_key"]].fillna("").replace("fast", "orig").merge(
    addresses_batch2[["city", "postcode","street", "housenumber", "method", "addr_out_street", "addr_out_number", "addr_out_postcode", "addr_out_city", "addr_key"]].fillna(""), how="outer", indicator=True)
if mg[mg._merge == "both"].shape[0] == addresses.shape[0]:
    print("Same result in seq and dask run!")
else: 
    print("!!! Not the same result in seq and dask run!")
    

Same result in seq and dask run!


In [318]:
mg[mg._merge != "both"].sort_values("addr_key")

,city,postcode,street,housenumber,method,addr_out_street,addr_out_number,addr_out_postcode,addr_out_city,addr_key,_merge


## Comparing options

In [ ]:
addresses = get_addresses("address.csv.gz")
addresses = addresses[addresses[country_field] == "Belgique"]
addresses = addresses.sample(10000).copy()

In [ ]:
results = {}
it_per_seconds=pd.DataFrame()

for check_label in ["check", "nocheck"]:
    for struct_label in ["struct", "unstruct" ]:
        print(check_label, struct_label)
        start=datetime.now()
        
        results[(check_label, struct_label)] = call_ws_batch_chunks(addresses, 
                                                                    mode="short", 
                                                                    check_result   =  check_label == "check", 
                                                                    structured_osm =  struct_label == "struct")
        
        it_per_seconds.loc[check_label, struct_label] = addresses.shape[0] / (datetime.now()-start).total_seconds()
print("Iterations per seconds:")
it_per_seconds

In [ ]:
print("Match rate")
pd.DataFrame({k1: {k2: results[(k1,k2)].shape[0]/addresses.shape[0] for k2 in ["struct", "unstruct"]} 
                  for k1 in  ["check","nocheck"]})

In [ ]:
print("Match rate (without nostreet)")
pd.DataFrame({k1: {k2: results[(k1,k2)].query("method!='nostreet'").shape[0]/addresses.shape[0] for k2 in ["struct", "unstruct"]} 
                  for k1 in  ["check","nocheck"]})

In [ ]:
print("Unmatched addresses")
for k1 in results:
    print(k1)
    nomatch=addresses[~addresses[addr_key_field].isin(results[k1]["addr_key"])]
    display(nomatch)
    print(nomatch[country_field].value_counts())

In [ ]:
vc_values = pd.DataFrame(columns=results.keys(), index=results.keys())

for k1 in results:
    vc_values.loc[k1, k1] = results[k1].shape[0]
    for k2 in results:
        if k1>k2:
            r1=results[k1]
            r2=results[k2]
            mg = r1[["addr_key", "place_id"]].merge(r2[["addr_key", "place_id"]], on="addr_key", how="outer", indicator=True)
 
            vc = mg._merge.value_counts()

            mismatches = mg[mg.place_id_x != mg.place_id_y][["addr_key"]]
            mismatches = mismatches.merge(addresses.rename({addr_key_field:"addr_key"}, axis=1))
            mismatches = mismatches.merge(r1[["addr_key", "addr_out_street", "addr_out_number", "extra_house_nbr", "addr_out_postcode", "addr_out_city"]], on="addr_key")
            mismatches = mismatches.merge(r2[["addr_key", "addr_out_street", "addr_out_number", "extra_house_nbr", "addr_out_postcode", "addr_out_city"]], on="addr_key")
            mismatches.columns = pd.MultiIndex.from_arrays([["Input"]*6 + [f"x:{k1}"]*5 + [f"y:{k2}"]*5, mismatches.columns])

            mismatch_values = mismatches[(mismatches[f"x:{k1}"].rename(lambda x: x.replace("_x", ""), axis=1).fillna("") != 
                                          mismatches[f"y:{k2}"].rename(lambda x: x.replace("_y", ""), axis=1).fillna("")).any(axis=1)]
            
            mismatch_values_no_nmbr = mismatches[(mismatches[f"x:{k1}"].rename(lambda x: x.replace("_x", ""), axis=1).drop("addr_out_number", axis=1).fillna("") != 
                                                  mismatches[f"y:{k2}"].rename(lambda x: x.replace("_y", ""), axis=1).drop("addr_out_number", axis=1).fillna("")).any(axis=1)]
            
            
            vc_label = f"{vc['both']} ({mismatches.shape[0]} - {mismatch_values.shape[0]} - {mismatch_values_no_nmbr.shape[0]}) / {vc['left_only']} / {vc['right_only']}"
            vc_values.loc[k1, k2]=vc_label

                
            print(f"{k1} vs {k2}")
            print(vc_label)
            print("-----------------------------")
            
            print(f"Only in {k1}")
            display(r1[r1.addr_key.isin(mg[mg._merge=="left_only"].addr_key)].merge(addresses.rename({addr_key_field:"addr_key"}, axis=1)))
            
            print(f"Only in {k2}")
            display(r2[r2.addr_key.isin(mg[mg._merge=="right_only"].addr_key)].merge(addresses.rename({addr_key_field:"addr_key"}, axis=1)))
            
            print("Mismatch on place_id")
            display(mismatches)
            
            print("Mismatch on values")
            
            display(mismatch_values)
            
            print("Mismatch on values (no nbr)")
            display(mismatch_values_no_nmbr)
            
            print("#######################")
            
# display(vc_values.fillna(""))

In [ ]:
print("Common in both (disagree on place_id - disagree on values - disagree on values, ignoring number) / results only for row / results only for columns")
vc_values.fillna("")

# tests

In [100]:
osm_host ="172.27.0.64:8080"
def get_osm(addr, accept_language = ""): #lg = "en,fr,nl"
    params = urllib.parse.urlencode({"q": addr,
                                    "format":"jsonv2",
                                    "accept-language":accept_language,
                                    "addressdetails":"1",
                                    "namedetails" : "1",
                                    "limit": "50"
                                    })
    
    url = "http://%s/search.php?%s"%(osm_host, params)
#     print(f"Call to OSM: {url}")
    try: 
        with urllib.request.urlopen(url) as response:
            res = response.read()
            res = json.loads(res)
#             return res
            return [ {field: item[field] for field in ["place_id", "lat", "lon", "display_name", "address", "namedetails", "place_rank", "category", "type"]} for item in res] 
    except Exception as e:
        raise Exception (f"Cannot get OSM results ({osm_host}): {e}") 

In [322]:
%timeit get_osm("Av. Fonsny 20, 1060 Bruxelles")

10.2 ms ± 100 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [323]:
%timeit call_ws_test({street_field:   "Av. Fonsny", \
         housenbr_field: "20",\
         city_field:     "Saint-Gilles",\
         postcode_field: "1060",\
         country_field:  "Belgium"}, check_result=False, structured_osm=False)
# res

1.09 ms ± 17.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
